['absolutely' 'ever' 'experience' 'fantastic' 'is' 'love' 'product'
 'service' 'the' 'this' 'worst']


In [3]:
background_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [4]:


# Simulate a binary mask: e.g., keep only 'love' and 'product'
keep = ["love", "product"]
mask = np.array([1 if word in keep else 0 for word in feature_names])



In [5]:
mask

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [7]:
# Create a perturbed version: masked features replaced by background
perturbed_vector = np.where(mask == 1, instance_vector, background_vector)
perturbed_vector

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.62276601, 0.62276601, 0.        , 0.        , 0.        ,
       0.        ])

In [8]:


# Show side-by-side
import pandas as pd

df = pd.DataFrame({
    "feature": feature_names,
    "original_tfidf": instance_vector,
    "mask": mask,
    "perturbed_tfidf": perturbed_vector
})

df

feature  original_tfidf  mask  perturbed_tfidf
0   absolutely        0.000000     0         0.000000
1         ever        0.000000     0         0.000000
2   experience        0.000000     0         0.000000
3    fantastic        0.000000     0         0.000000
4           is        0.000000     0         0.000000
5         love        0.622766     1         0.622766
6      product        0.622766     1         0.622766
7      service        0.000000     0         0.000000
8          the        0.000000     0         0.000000
9         this        0.473630     0         0.000000
10       worst        0.000000     0         0.000000

In [11]:

df = df[(df["original_tfidf"] > 0) | (df["perturbed_tfidf"] > 0)]
print(df.sort_values("original_tfidf", ascending=False))

# Predict
original_pred = model.predict_proba([instance_vector])[0,1]
perturbed_pred = model.predict_proba([perturbed_vector])[0,1]
delta = original_pred - perturbed_pred

print(f"\nOriginal Prediction: {original_pred:.4f}")
print(f"Perturbed Prediction (only 'love' and 'product'): {perturbed_pred:.4f}")
print(f"Change in prediction (contribution of masked features): {delta:.4f}")

   feature  original_tfidf  mask  perturbed_tfidf
5     love        0.622766     1         0.622766
6  product        0.622766     1         0.622766
9     this        0.473630     0         0.000000

Original Prediction: 0.7161
Perturbed Prediction (only 'love' and 'product'): 0.7204
Change in prediction (contribution of masked features): -0.0042


In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

print(es.info())

{'name': 'a11a2100a8aa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'gAf831kRS1GKH2EiTlG_ug', 'version': {'number': '8.13.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '16cc90cd2d08a3147ce02b07e50894bc060a4cbf', 'build_date': '2024-04-05T14:45:26.420424304Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
